In [21]:
L = !ls ../data/assorted_classical/classical_piano_C
m = midiread("../data/assorted_classical/classical_piano_C/{}".format(L[1]))
dir(m)

['__doc__',
 '__init__',
 '__module__',
 '_absolute_time',
 '_current_track',
 '_relative_time',
 '_tempo',
 'abs_time',
 'abs_time_in_seconds',
 'active_sensing',
 'aftertouch',
 'beat',
 'channel_message',
 'channel_pressure',
 'continuous_controller',
 'copyright',
 'cuepoint',
 'device_name',
 'div',
 'end_of_track',
 'eof',
 'get_current_track',
 'get_run_stat',
 'header',
 'instrument_name',
 'key_signature',
 'lyric',
 'marker',
 'meta_event',
 'midi_ch_prefix',
 'midi_port',
 'midi_time_code',
 'note_off',
 'note_on',
 'notes',
 'patch_change',
 'piano_roll',
 'pitch_bend',
 'rel_time',
 'reset_run_stat',
 'reset_time',
 'sequence_name',
 'sequence_number',
 'sequencer_specific',
 'set_current_track',
 'set_run_stat',
 'smtp_offset',
 'song_continue',
 'song_position_pointer',
 'song_select',
 'song_start',
 'song_stop',
 'start_of_track',
 'sysex_event',
 'system_exclusive',
 'system_reset',
 'tempo',
 'text',
 'time',
 'time_signature',
 'timing_clock',
 'tuning_request',
 'u

In [24]:
help(midiwrite)

Help on function midiwrite in module utils:

midiwrite(filename, piano_roll, r=(21, 109), dt=0.2, patch=0)



In [10]:
%matplotlib inline
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
sys.path.append("../midi")

from utils import midiread, midiwrite
sys.path.append("/Users/danshiebler/Documents")
import helper_functions as hf
# sys.path.append('/home/hanhong/Projects/python27/DeepLearningTutorials/code/')
# from utils import tile_raster_images
# import Image
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
from matplotlib import pyplot as plt
import glob
from copy import deepcopy
from tensorflow.examples.tutorials.mnist import input_data

def get_songs(path='../data/'):
    files = glob.glob(path)[:5]
    f = files[0]
    dt=0.3
    songs = []
    for f in tqdm(files):
        try:
            songs.append(midiread(f, r, dt).piano_roll)
        except Exception as e:
            print e            
    return songs

songs = get_songs()


100%|██████████| 1/1 [00:00<00:00, 6096.37it/s]

[Errno 21] Is a directory: '../data/'


In [ ]:

def build_rbm(x, W, bv_t, bh_t, k):
    W = tf.Print(W, [W], "W")
    bv_t = tf.Print(bv_t, [bv_t], "bv_t")
    bh_t = tf.Print(bh_t, [bh_t], "bh_t")

    size_bt = tf.shape(x)[0]

    # helper functions
    def sample(probs):
        return tf.to_float(tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1)))

    def sampleInt(probs):
        return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

    # define graph/algorithm

    # CD-k
    # we use tf.while_loop to achieve the multiple (k - 1) gibbs sampling  
    # set up tf.while_loop()
    def rbmGibbs(xx, hh, count, k):
        xk = sampleInt(tf.sigmoid(tf.matmul(hh, tf.transpose(W)) + tf.tile(bv_t, [size_bt, 1])))
        hk = sampleInt(tf.sigmoid(tf.matmul(xk, W) + tf.tile(bh_t, [size_bt, 1])))
        return xk, hk, count+1, k

    def lessThanK(xk, hk, count, k):
        return count <= k

    ct = tf.constant(1) #counter
    h = sample(tf.sigmoid(tf.matmul(x, W) + tf.tile(bh_t, [size_bt, 1])))# sample h x1 h1 ..
    [xk1, hk1, _, _] = control_flow_ops.While(lessThanK, rbmGibbs, [x, h, ct, tf.constant(k)], 1, False)

    def free_energy(xx):
        #return -(v * bv_t).sum() - T.log(1 + T.exp(T.dot(v, W) + bh_t)).sum()
        A = -tf.reduce_sum(tf.sigmoid(tf.matmul(xx, W) + tf.tile(bh_t, [size_bt, 1])), 1)
        B = -tf.matmul(xx, tf.transpose(tf.cast(tf.tile(bv_t, [size_bt, 1]), tf.float32)))
        return tf.add(B, A)

    #Define loss and optimizer
    #the cost is based on the difference in free energy between v and v_sample
    e1 = free_energy(x)
    e2 = free_energy(xk1)
    cost = tf.reduce_mean(tf.sub(e1, e2))
    return xk1, cost


In [ ]:
def build_rnnrbm( n_visible= 88, n_hidden= 150, n_hidden_recurrent= 100):

    # variables and place holder
    x  = tf.placeholder(tf.float32, [None, n_visible])
    a  = tf.placeholder(tf.float32)
    size_bt = tf.shape(x)[0]

    W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01))
    bh = tf.Variable(tf.zeros([n_hidden], tf.float32))
    bv = tf.Variable(tf.zeros([n_visible], tf.float32))
    Wuh = tf.Variable(tf.random_normal([n_hidden_recurrent, n_hidden], 0.0001))
    Wuv = tf.Variable(tf.random_normal([n_hidden_recurrent, n_visible], 0.0001))
    Wvu = tf.Variable(tf.random_normal([n_visible, n_hidden_recurrent], 0.0001))
    Wuu = tf.Variable(tf.random_normal([n_hidden_recurrent, n_hidden_recurrent], 0.0001))
    bu  = tf.Variable(tf.zeros([n_hidden_recurrent],  tf.float32))
    
#     hf.variable_summaries(W, "W")
#     hf.variable_summaries(bh, "bh")
#     hf.variable_summaries(bv, "bv")
#     hf.variable_summaries(Wuh, "Wuh")
#     hf.variable_summaries(Wuv, "Wuv")
#     hf.variable_summaries(Wvu, "Wvu")
#     hf.variable_summaries(Wuu, "Wuu")
#     hf.variable_summaries(bu, "bu")

    params = W, bh, bv, x, a, Wuh, Wuv, Wvu, Wuu, bu

    # If `v_t` is given, deterministic recurrence to compute the variable
    # biases bv_t, bh_t at each time step. If `v_t` is None, same recurrence
    # but with a separate Gibbs chain at each time step to sample (generate)
    # from the RNN-RBM. The resulting sample v_t is returned in order to be
    # passed down to the sequence history.
    
    def recurrence(xx, u_tm1, _0, _1, count, k):
        c = tf.Print(Wuv, [Wuv], "Wuv")
        c = tf.Print(Wuh, [Wuh], "Wuh")

        bv_t = tf.add(bv, tf.matmul(u_tm1, Wuv))
        bh_t = tf.add(bh, tf.matmul(u_tm1, Wuh))
        
        sl   = tf.slice(xx, [count, 0], [1, n_visible])
        u_t  = tf.tanh(bu + tf.matmul(sl, Wvu) + tf.matmul(u_tm1, Wuu))
        return [xx, u_t, bv_t, bh_t, count+1, k]

    def lessThanNumIter(_0, _1, _2, _3, count, num_iter):
        return count < num_iter

    ct   = tf.constant(1, tf.int32) #counter
    u0   = tf.zeros([1, n_hidden_recurrent], tf.float32)
    bh_t = tf.zeros([1, n_hidden],  tf.float32)#tf.random_uniform([1, n_hidden], -0.005, 0.005)
    bv_t = tf.zeros([1, n_visible],  tf.float32)#tf.random_uniform([1, n_visible], -0.005, 0.005)
    [x, u_t, bv_t, bh_t, _, _] = control_flow_ops.While(lessThanNumIter, recurrence, [x, u0, bv_t, bh_t, ct, size_bt])

    #Build this rbm based on the bias vectors that we already found 
    sample, cost = build_rbm(x, W, bv_t, bh_t, k=15)
    return x, sample, cost, params, bv_t, bh_t, size_bt

x, sample, cost, params, bv_t, bh_t, size_bt = build_rnnrbm()

W, bh, bv, x, a, Wuh, Wuv, Wvu, Wuu, bu = params
optimizer = tf.train.AdamOptimizer(learning_rate=a).minimize(cost)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


In [ ]:

num_epochs = 10
batch_size = 100

# sess.close()
sess =  tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter('/tmp/tf_rbm_logs/train', sess.graph)

# loop with batch
for epoch in tqdm(range(num_epochs)):
    for song in songs[:5]:
        for i in range(1, len(song), batch_size):
            tr_x = song[i:i + batch_size]
#             alpha = min(0.05, 100/float(i))
            summary, _ = sess.run([merged, optimizer], feed_dict={x: tr_x, a: 0.1})
            writer.add_summary(summary, i)



In [ ]:
# before_W   = deepcopy(W.eval(session=sess))
# before_Wuh = deepcopy(Wuh.eval(session=sess))
# before_Wuv = deepcopy(Wuv.eval(session=sess))
# before_Wvu = deepcopy(Wvu.eval(session=sess))
# before_Wuu = deepcopy(Wuu.eval(session=sess))

# before_bh  = deepcopy(bh.eval(session=sess))
# before_bv  = deepcopy(bv.eval(session=sess))
# before_bu  = deepcopy(bu.eval(session=sess))

# for wb, w in zip([before_W, before_Wuh, before_Wuv, before_Wvu, before_Wuu], [W, Wuh, Wuv, Wvu, Wuu]):
#     plt.figure()
#     plt.subplot(121)
#     hf.plot2d(wb)
#     plt.subplot(122)
#     hf.plot2d(w.eval(session=sess))
    
# for bb, b in zip([before_bh, before_bv, before_bu], [bh, bv, bu]):
#     plt.figure()
#     plt.subplot(121)
#     plt.plot(bb)
#     plt.subplot(122)
#     plt.plot(b.eval(session=sess))

In [ ]:
sess.close()
